# Day in the Life of a Data Scientist
### J. Andrew Howe, PhD

- <a href=#bapara>BAPARA</a>
- <a href=#acquire>**Acquire**</a>
- <a href=#prepare-explore>**Prepare**-Explore</a>
- <a href=#prepare-preprocess>**Prepare**-Preprocess</a>
- <a href=#analyze-hypothesize>**Analyze**-Hypothesize</a>
- <a href=#analyze-select>**Analyze**-Select</a>
- <a href=#analyze-build>**Analyze**-Build</a>
    - <a href=#linreg>Multiple Linear Regression</a>
    - <a href=#dectree>Decision Tree Regression</a>
    - <a href=#nn>Simlple Neural Network</a>
- <a href=#analyze-assess>**Analyze**-Assess</a>
- <a href=#report-act>**Report**</a>
- <a href=#report-act>**Act**</a>
- <a href=#end>The End</a>

<a id=top></a>

This notebook will summarize the basic process and several of the stages of the data science process, essentially capturing a day in the life of a data scientist or statistical modeler.

We use a simple "toy" dataset for demonstrative purposes, without loss of much generality.

[This notebook](https://github.com/ahowe42/MachineLearningKernelsDemo) Goes into much more detail on several machine learning concepts:
- Exploratory Data Analysis & Visualization
- Feature Engineering
- Supervised Classification Modeling
- Logistic Regression
- Numeric Optimization
- Support Vector Machines
- Reproducing Kernel Hilbert Spaces
- Cross-Validation
- Hyperparameter Tuning
- Feature Selection
- Feature Hierarchy
- Genetic / Evolutionary Algorithms
- Information Theoretic Criteria
- Ensemble Modeling

## Imports & Definitions
<a id=impdef></a>

In [ ]:
import ipdb
import math
import numpy as np
import pandas as pd
from copy import deepcopy
from captum.attr import DeepLift, IntegratedGradients, NoiseTunnel

from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit, ShuffleSplit

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

import chart_studio.plotly as ply
import plotly.offline as plyoff
import plotly.graph_objects as go
import plotly.subplots as plysub
import plotly.express as px
import plotly.tools as plytool

# to use plotly offline, need to initialize with a plot
plyoff.init_notebook_mode(connected=True)
init = go.Figure(data=[go.Scatter({'x':[1, 2], 'y':[42, 42], 'mode':'markers'})],
                 layout=go.Layout(width=42, height=42, title='Init', xaxis={'title':'x'}))
plyoff.iplot(init)

pd.options.display.max_rows = None
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None

In [ ]:
# define my own RMSE metric
def RMSE(y_true, y_pred):
    return math.sqrt(mean_squared_error(y_true, y_pred))

In [ ]:
# plotting function for correlations
def correlationsPlot(rhoYourBoat, plotTitl='Feature Correlations Plot', trcLims=(0.0, 1.0), tweaks=(20, None, None, 1.1)):
    '''
    This creates and returns a bubble plot visualization of a correlation matrix. The
    sizes of the bubbles are proportional to the absolute magnitude of the correlations.
    Positive correlations are only plotted in the upper triangle, with colors ranging
    from green (0) to red (+1). Negative correlations are plotted in the lower triangle,
    with colors ranging from green (0) to blue (-1). Perfect correlations are indicated
    with black bubbles.
    :param rhoYourBoat: dataframe of correlation matrix, probably created with pd.DataFrame.corr()
    :param plotTitl: optional (default='Feature Correlations Plot') plot title
    :param trcLims: (default=(0.0,1.0)) = ordered tuple of "buckets" in which to place absolute correlations for
        plotting traces; must include at least 0 and 1
    :param tweaks (default=(20,None,None,1.1)): tuple of position & size tweaking values for plotly; maximum size of
        bubbles, plot width, plot height, y position of legend
    :return fig: plotly correlation plot figure
    '''

    # set the granulatrity of the colors
    n = 101  # must be odd so in the middle at correlation = 0 is just green

    # number features
    p = len(rhoYourBoat.columns)
    ps = list(range(p))

    # positive correltions are red>green
    scl = np.linspace(1.0, 0.0, n)
    redsP = np.round(255 * scl)
    grnsP = 255 - redsP
    blusP = [0.0] * n

    # negative correlations are blue>green
    scl = scl[:-1]
    blusN = np.round(255 * scl)
    grnsN = 255 - blusN
    redsN = [0.0] * n

    # adding 2 more to make the endpoints for perfect correlations
    scl = np.linspace(-1.0, 1.0, 2 * n - 1 + 2)

    # make the colormap - perfectly uncorrelated and perfectly correlated are black
    rgb = ['rgb(0,0,0)']
    rgb.extend(['rgb(%d,%d,%d)' % (r, g, b) for r, g, b in
                zip(np.r_[redsN, redsP[::-1]], np.r_[grnsN, grnsP[::-1]], np.r_[blusN, blusP[::-1]])])
    rgb.append('rgb(0,0,0)')

    # now map correlations to colors - unhappy that I have to do this double loop :-(
    vals = rhoYourBoat.values
    cols = np.zeros(shape=vals.shape, dtype=object)
    for i in ps:
        for j in ps:
            v = vals[i, j]
            mni = np.argmin(np.abs(v - scl))
            mnv = scl[mni]
            cols[i, j] = rgb[mni]
            # print('%0.5f,%d,%0.5f,%s'%(v,mni,mnv,cols[i,j]))

    # filter data so the upper triangle is (+) correlations and lower triangle is (-) correlations
    y = np.tile(ps, (p, 1))
    x = y.T
    x = x.flatten()
    y = y.flatten()
    vals = vals.flatten()
    cols = cols.flatten()
    keepind = ((y > x) & (vals > 0)) | ((x > y) & (vals < 0))
    x = x[keepind]
    y = y[keepind]
    vals = vals[keepind]
    cols = cols[keepind]
    absVals = np.abs(vals)

    # set a minimum bubble size
    minBub = 0.09 * tweaks[0]

    # put together the figure - make multiple traces
    trc = [go.Scatter(
        {'x': ps, 'y': ps, 'mode': 'lines', 'line': {'color': 'black'}, 'showlegend': False, 'hoverinfo': 'skip'})]
    for i, t in enumerate(trcLims):
        # build the index for the traces
        if i == 0:
            continue
        elif i == 1:
            indx = (absVals <= t) & (absVals >= trcLims[0])
            trcName = '$\\vert\\rho\\vert\\in[%0.2f,%0.2f]$' % (trcLims[0], t)
        else:
            indx = (absVals <= t) & (absVals > trcLims[i - 1])
            trcName = '$\\vert\\rho\\vert\\in(%0.2f,%0.2f]$' % (trcLims[i - 1], t)
        # create & add the trace
        trc.append(go.Scatter({'x': x[indx], 'y': y[indx], 'mode': 'markers', 'text': ['%0.4f' % v for v in vals[indx]],
                               'name': trcName, 'hoverinfo': 'x+y+text',
                               'marker': {'color': cols[indx], 'line': {'color': cols[indx]},
                                          'size': np.maximum(tweaks[0] * absVals[indx], minBub)}}))

    # finalize the layout
    lout = go.Layout({'title': plotTitl, 'legend': {'orientation': 'h', 'x': 0, 'y': tweaks[-1]},
                      'xaxis': {'ticklen': 1, 'tickvals': ps, 'ticktext': rhoYourBoat.columns.values,
                                'mirror': True, 'showgrid': False, 'range': [-1, p], 'linecolor': 'black',
                                'linewidth': 0.5, 'zeroline': False, 'tickangle': 90},
                      'yaxis': {'ticklen': 1, 'tickvals': ps, 'ticktext': rhoYourBoat.index.values,
                                'mirror': True, 'showgrid': False, 'range': [-1, p], 'linecolor': 'black',
                                'linewidth': 0.5, 'zeroline': False}})
    if tweaks[1] is not None:
        lout['width'] = tweaks[1]
    if tweaks[2] is not None:
        lout['height'] = tweaks[2]

    return go.Figure(data=trc, layout=lout)

In [ ]:
# plotting function for common regression diagnostic plots
def ResultsPlots(data, sequenceCol, responseCol, predCol, resdCol, colorCol, overall_title, plot_colors=('red',)*4):
    '''
    This creates and returns a quad-plot of results from a model. The four plots are: a) prediction vs response,
    b) histogram of residuals, c) residuals by sequence, d) residuals by response. They are arranged as
    [[a,b],[c,d]].
    :param data: dataframe holding the sequence, response, prediction, and residual columns
    :param sequenceCol: column in the dataframe holding the time or sequence counter; if None, a counter will be used
    :param responseCol: column in the dataframe holding the response variable
    :param predCol: column in the dataframe holding the model predictions
    :param resdCol: column in the dataframe holding the model residuals
    :param colorCol: optional column in the dataframe holding the color for each observation for the plots
    :param overall_title: title to go on top of the quad plot
    :param plot_colors (default=(red, red, red, red)): optional tuple of colors for each plot; only used if colorCol
    	not passed
    :return fig: plotly plot figure
    '''
    
    # copy the input dataframe
    data = data.copy(deep=True)
    
    if colorCol is None:
    	colorCol = 'NOCOLOR'
    
    # setup the subplot
    figRes = plysub.make_subplots(rows=2, cols=2, subplot_titles=['Predictions vs Responses', 'Residuals Distribution','Residuals by Sequence', 'Residuals vs Responses'])
    
    # for the actual vs preds plot, build the fit line
    actual = data[responseCol].values.reshape(-1,1)
    lindat = np.linspace(actual.min(), actual.max(), 10).reshape(-1, 1)
    fitlin = LinearRegression(n_jobs=-1)
    fitlin.fit(X=actual, y=data[predCol])
    actpred = fitlin.predict(X=lindat)
    r2 = fitlin.score(X=actual, y=data[predCol])
    # create the trace and annotation
    r2trc = go.Scatter(x=lindat.squeeze(), y=actpred, mode='lines', name='fit', line={'color':'black','width':1}, showlegend=False)
    r2ann = dict(x=lindat[5][0], y=actpred[5], xref='x1', yref='y1', text='$\\rho=%0.3f$'%(r2), showarrow=False, bgcolor='#ffffff')
    
    # actuals vs resids plot
    if colorCol == 'NOCOLOR':
        data[colorCol] = plot_colors[0]
    figRes.add_trace(go.Scatter(x=data[responseCol], y=data[predCol], mode='markers', marker={'color':data[colorCol]}, showlegend=False), 1,1)
    figRes.add_trace(r2trc, 1, 1)
    figRes['layout']['xaxis1'].update(title=responseCol)
    figRes['layout']['yaxis1'].update(title=predCol)
    
    # residuals histogram
    if colorCol == 'NOCOLOR':
        data[colorCol] = plot_colors[1]
    figRes.add_trace(go.Histogram(x=data[resdCol], histnorm='', marker={'color':data[colorCol]}, showlegend=False), 1,2)
    figRes['layout']['xaxis2'].update(title=resdCol)
    figRes['layout']['yaxis2'].update(title='count')
    
    # get the time variable
    if sequenceCol is None:
        seq = list(range(len(data)))
        seqNam = 'sequence'
    else:
        seq = data[sequenceCol]
        seqNam = sequenceCol
    
    # residuals by time plot
    if colorCol == 'NOCOLOR':
        data[colorCol] = plot_colors[2]
    figRes.add_trace(go.Scatter(x=seq, y=data[resdCol], mode='markers', marker={'color':data[colorCol]}, showlegend=False), 2, 1)
    figRes['layout']['xaxis3'].update(title=seqNam)
    figRes['layout']['yaxis3'].update(title=resdCol)
    
    # residuals by response plot
    if colorCol == 'NOCOLOR':
        data[colorCol] = plot_colors[3]
    figRes.add_trace(go.Scatter(x=data[responseCol], y=data[resdCol], mode='markers', marker={'color':data[colorCol]}, showlegend=False), 2,2)
    figRes['layout']['xaxis4'].update(title=responseCol)
    figRes['layout']['yaxis4'].update(title=resdCol)
    
    # update layout
    figRes['layout'].update(title=overall_title, height=1000)
    anns = list(figRes['layout']['annotations'])
    anns.append(r2ann)
    figRes['layout']['annotations'] = anns
    
    return figRes

In [ ]:
# define the model class for a neural net with variable hidden layers & dropout
class myNN(nn.Module):
    def __init__(self, layerNodes, wInit, pDropout, activations):
        super(myNN, self).__init__()
        self.activations = activations[1:]
        self.len = len(layerNodes)-1
        self.drop = nn.Dropout(pDropout)
        self.linears = nn.ModuleList()
        for I, O in zip(layerNodes, layerNodes[1:]):
            # create the layer
            lin = nn.Linear(I, O)
            # initialize it
            nn.init.zeros_(lin.bias)
            if wInit == 'uni':
                nn.init.uniform_(lin.weight)
            elif wInit == 'xav':
                nn.init.xavier_uniform_(lin.weight)
            elif wInit == 'kai':
                nn.init.kaiming_uniform_(lin.weight, nonlinearity='relu')
            # and now add it
            self.linears.append(lin)

    def forward(self, x):
        for i, (L, A) in enumerate(zip(self.linears, self.activations)):
            # dropout if not the output layer
            if i <= self.len - 1:
                x = self.drop(L(x))
            else:
                x = L(x)
            # compute the activation
            if A == 'relu':
                x = nn.functional.relu(x)
            elif A == 'sigmoid':
                x = torch.sigmoid(x)
            elif A == 'leakyrelu':
                x = nn.functional.leaky_relu(x)
            elif A == 'tanh':
                x = torch.tanh(x)
        return x
    
    def __str__(self):
        mn = super(myNN, self).__str__()
        return '%s\nActivations: %s'%(mn, self.activations)
    
    def show(self):
        print('Weights')
        for lin in self.linears:
            print(lin.weight.detach().numpy())
        print('Biases')
        for lin in self.linears:
            print(lin.bias.detach().numpy())        

In [ ]:
# function to train an nn, based on a loss and optimizer
def trainNN(data, network, loss, optimizer, scheduler, epochs, writer, randState=None, talkFreq=0.2):
    '''
    Train a neural network specified by a network, optimizer, and loss, fitting to data from
    a training dataloader, and evaluating on a testing dataloader. NB This procedure updates
    the input network *in place*.
    :param data: list holding the training set data loader and testing set dataloader. The first
        may load in batches, while the second may not
    :param network: PyTorch NN architecture as defined to extend the nn.Module class, or using
        the Sequential constructor
    :param loss: PyTorch loss function
    :param optimizer: PyTorch optimizer
    :param scheduler: PyTorch learning rate decay scheduler
    :param epochs: integer number of epochs
    :param writer: PyTorch TensorBoard SummaryWriter object
    :param randState: optional seed for PyTorch psuedo random number generator
    :param talkFreq: optional (default=0.2) frequency with which progress should be printed
    :return trnLoss: loss on training set per epoch
    :return tstLoss: loss on testing set per epoch
    '''
    
    # set the prng seed, maybe
    if randState:
        torch.manual_seed(randState)
        writer.add_scalar('Params/random_state', randState, 0)
        
    # save the learning rate
    writer.add_scalar('Params/initial_learning_rate', optimizer.param_groups[0]['initial_lr'], 0)
    
    # add the model graph
    try:
        writer.add_graph(network, data[0].dataset.x)
    except TypeError as err:
        print("Network may be from nn.Sequential, so can't be added to TensorBoard!")
        
    # get the data loaders
    trn, tst = data
    
    # containers for training / testing loss
    trnLoss = [np.inf]*epochs
    tstLoss = [np.inf]*epochs

    # iterate over epochs
    for epoch in range(epochs):
        # train with minibatch gradient descent
        network.train(True) # setting train to True tells pytorch that ops like dropout / batch normalization to occur, which wouldn't occur during evaluation
        # iterate over batches
        for indx, (x, y) in enumerate(trn):
            # forward step
            yhat = network(x) # implements forward propagation as implemented by the forward() method
            # compute loss (not storing for now, will do after minibatching)
            l = loss(yhat, y)
            # backward step
            optimizer.zero_grad() # set gradients to zero before backprop, so there's no accumulation among batches
            l.backward() # backward propagation computes the gradients of the parameters
            optimizer.step()
            
        # update the learning rate
        scheduler.step()
        writer.add_scalar('Params/learning_rate', scheduler.get_last_lr()[0], epoch)
        
        # evaluate performance
        with torch.no_grad():
            network.train(False) # using no_grad() and train() false turns off any gradient updating or special functionality
            # evaluate loss on training set
            yhat = network(trn.dataset.x)
            trnLoss[epoch] = loss(yhat, trn.dataset.y)
            # evaluate loss on testing set
            yhat = network(tst.dataset.x)
            tstLoss[epoch] = loss(yhat, tst.dataset.y)
            # tensorboard
            writer.add_scalar('Train/Loss', trnLoss[epoch], epoch)
            writer.add_scalar('Test/Loss', tstLoss[epoch], epoch)
            # maybe talk
            if epoch % (epochs*talkFreq) == 0:
                print('Epoch %d Training (Testing) Loss = %0.2f (%0.2f)'%(epoch, trnLoss[epoch], tstLoss[epoch]))

    print('==========\nTraining Initial Loss = %0.2f, Final Loss = %0.2f'%(trnLoss[0], trnLoss[-1]))
    print('Testing Initial Loss = %0.2f, Final Loss = %0.2f'%(tstLoss[0], tstLoss[-1]))
    
    # return results
    return trnLoss, tstLoss

In [ ]:
# define the data class
class Data(Dataset):
    def __init__(self, x, y):
        self.x = torch.FloatTensor(x)
        self.y = torch.Tensor(y.astype(int))
        self.len, self.p = self.x.shape
    def __getitem__(self, index):      
        return self.x[index], self.y[index]
    def __len__(self):
        return self.len

## BAPARA
For analytics projects, we follow a modification of the most widely-used process model: [***CRISP-DM***](https://en.wikipedia.org/wiki/Cross-industry_standard_process_for_data_mining). CRISP-DM is a high-level process model that iterates over 6 phases, shown here:<br><img src="./CRISP-DM_Process_Diagram.png" alt="CRISP-DM" width="300" height="300"><br>A fundamental component that is often overlooked while performing analytics, or at least only included implicitly, is business understanding. The extent to which any analytics project is impactful (*"moves the needle"*, according to our founder Preston Cody) is highly dependent on accurate understanding of the business environment and need. Digging deeper into this high-level process model - which also specifies lower-level stages - the Data Understanding through Evaluation phases often iterate and loop back more than is shown by the CRISP-DM process diagram.

Our modification of CRISP-DM is called ***BAPARA***; the process is composed of 13 segments of 6 phases, defined below, which can loop back at multiple points in the process, as shown here:<br><img src="BAPARA_Process_DiagramW.png" alt="BAPARA" height="200">

In the remainder of this discussion, we will detail and demonstrate most of these phases.

<a href=#top>Go to top</a>
<a id=bapara></a>

## Business Understanding
Accurate and complete business and domain understanding is necessary for almost all segments of the BAPARA process.
- Embed an SME: in addition to technical and statistical expertise, project success relies on embedding a true "user" who is intimately involved with the impacted business workflow
- Explore Problem Space: business / workflow context of the problem to be solved and desired outcomes of the project must be defined; initial list of potential hypotheses to test can be determined; project success should be defined as quantitatively [***SMART***](https://en.wikipedia.org/wiki/SMART_criteria) as possible
- *Success Looks Like: The analytics team is staffed with a user, can articulate the desired outcomes, and has an initial list of hypotheses to consider. Can state a summary value proposition in the form of "For X, who want to Y, so they can Z".*

## Acquire - Identify, collect, and integrate relevant data
- Identify: consult with SMEs, seek out what is actually available, subject to constraints
- Collect: obtain appropriate access credentials, query structured / unstructured databases, access through business data warehouses, connect to web / network APIs, store raw accessed data - perhaps in a data lake or simple flat files
- Integrate: identify uniqueness and matching keys (primary & foreign keys in RDBMS parlance), join data sources appropriately (inner join, outer join, one-to-one, one-to-many, many-to-many)

<a href=#top>Go to top</a>
<a id=acquire></a>

- Many real-world data science problems require multiple datasets which must be joined, standardized, and jointly validated.
- Data can come from many sources: standalone files, automated feeds, databases, etc.
- Data can come in multiple formats as well: structured, geospatial, text, etc.
- Data acquisition & preparation generally taken to be about **80%** of a data science project.

For demonstrative purposes, we're just using the well-known Boston Housing dataset; this dataset is often used to test and benchmark performance on machine learning algorithms.

In [ ]:
# load and view the data
res = load_boston()
data = pd.DataFrame(data=np.c_[res['data'], res['target']], columns=res['feature_names'].tolist()+res.get('target_names', ['MedianHousePrice']))
(n, p) = data.shape
print(res['DESCR'])
display(data.head())

In [ ]:
# randomly permute observations, in case there is some order in the data
'''prng = 42
data = data.sample(frac=1.0, random_state=prng)'''

In [ ]:
# set data types - feature names are numpy.str instead of string, for some reason
cateFeatures = ['CHAS']
contFeatures = [str(feat) for feat in res['feature_names'] if feat not in cateFeatures]
response = 'MedianHousePrice'

## Prepare - Explore, clean, and pre-process acquired data
### Explore: descriptive statistics (probability distribution, centrality, dispersion, general morphology), visualizations, validation wrt domain expertise, evaluation of relationships between features (pairwise linear correlation, multiple regression), grouping structures, category frequencies & degree of imbalance; preparation of publication-ready statistical tables and data visualizations

<a href=#top>Go to top</a>
<a id=prepare-explore></a>

In [ ]:
# check descriptive statistics
print('Descriptive Statistics')
display(data[contFeatures].describe(percentiles=[0.05, 0.25, 0.5, 0.75, 0.95]))

#### Observations:
- scales: there is a wide range of scales between features; this should be handled by standardizing or normalizing features
- **CRIM**: the mean is much higher than the median, indicating positive skew or possible outliers; this is also seen in the difference between the 95th percentile and the max
- **ZN**: there is also a suggestion of skew or possible outliers

In [ ]:
# check correlations
print('Correlations')
display(data[['MedianHousePrice']+contFeatures].corr())

#### Observations:
- MedianHousePrice is only moderately *linearly* correlated with **RM** (positive) and **LSTAT** (negative)
- MedianHousePrice is only weakly correlated (linearly) with **AGE**, which is surprising
- It's not really easy to find high correlations between features in this table
- Also note that this only identifies *linear* correlations

In [ ]:
# categorical value counts
for feat in cateFeatures:
    print('%s Value Counts'%feat)
    featn = len(data[feat].values)
    vc = pd.DataFrame(data[feat].value_counts()).rename(columns={feat:'Frequency'})
    vc['Relative Frequency'] = vc['Frequency']/featn
    display(vc)

#### Observations:
- There is a high amount of imbalance in the Charles River flag feature.
- If **CHAS** is ultimately used in modeling, this imbalance will need to be taken into consideration.

In [ ]:
''' 5 x 3 subplots of histograms '''
# setup
fig = plysub.make_subplots(rows=5, cols=3, print_grid=False, subplot_titles=data.columns)
rowcols = [(math.floor(i/3)+1, i%3+1) for i in range(len(data.columns))]

# add traces
for (ind, feat) in enumerate(data.columns):
    r, c = rowcols[ind]
    fig.add_traces(go.Histogram(x=data[feat], histnorm='probability density'), r, c)

# update layout
fig['layout'].update({'title':'Feature Distributions', 'showlegend':False, 'height':1200})

# plot
plyoff.iplot(fig)

#### Observations
- **CRIM**: highly skewed positively, as suggested by descriptive statistics; most places have very low CRIM rates
- **ZN**: similarly skewed, with most areas having very low proportion of residences
- **RAD**: small percent of very high values; possible outliers to investigate
- **TAX**: tax rates show possible bimodal behavior; could perhaps be related to **CHAS**; need to investigate further
- **MedianHousePrice**: small group of very high-price regions; need to investigate if these are related to a feature; geospatial location could probably help predict this

In [ ]:
''' scatter plot matrix '''
# setup traces
trcs = [go.Splom(dimensions=[{'label':col, 'values':data.loc[data['CHAS']==0, col].values} for col in data.columns],
                showupperhalf=False, diagonal_visible=False, marker={'color':'red', 'opacity':0.4}, name='CHAS==0'),
        go.Splom(dimensions=[{'label':col, 'values':data.loc[data['CHAS']==1, col].values} for col in data.columns],
                showupperhalf=False, diagonal_visible=False, marker={'color':'green', 'opacity':0.4}, name='CHAS==1')]
fig = go.Figure(data=trcs, layout=go.Layout(title='Scatterplot Matrix', showlegend=True, height=1400))

# plot
plyoff.iplot(fig)

#### Observations
- a few curvilinear relationships between features visible: **MedianHousePrice** vs **LSTAT** and **DIS** vs **NOX** (possibly **INDUS**); these quadratic relationships can't be picked up by a linear model, without preprocessing of data
- only a few features seems to have linear (or nearly so) relationships with response - and these tend to have a lot of noise: **RM** and **NOX**
- some features also seem to have a relationship with the Charles River flag feature (see coloring): **DIS**, **B**, **CRIM**
- **MedianHousePrice**: would have expected grouping of prices by **CHAS**, but range of prices for regions not bound by the river entirely covers the range of prices for regions on the Charles river

In [ ]:
''' correlations plot '''
fig = correlationsPlot(data[['MedianHousePrice']+contFeatures].corr(), plotTitl='Feature Correlations Plot', trcLims=(0.0, 0.5, 0.75, 0.9, 0.95, 1.0), tweaks=(20, None, 1200, 1.1))
plyoff.iplot(fig)

#### Observations:
- Most features are only slightly linearly correlated.
- **DIS** and **NOX** have a moderately negative correlation.
- **INDUS** and **NOX** have a moderately positive correlation.
- **RAD** and **TAX** have a strong positive correlation - this might be strong enough for some people to drop one; however, reviewing the scatter plot matrix, this is likely due to the correlation fitting a line up to the single point that is an outlier for both features, so this is spurious.

### Clean: anomaly / outlier detection & handling, management of missing data (drop feature, drop rows, impute)

*skipping this segment*

### Pre-process - feature selection, feature extraction, feature engineering, normalization, encoding, discretization, interactions, other transformations; feature engineering should make use of domain knowledge to generate new features based on physical phenomena and relationships between features

<a href=#top>Go to top</a>
<a id=prepare-preprocess></a>

#### Explanation:
- Lacking any domain knowledge (other than *location, location, location*), we can't perform any "smart" features engineering.
- Given the characteristics observed in the plots, we see a reason to engineer 2 kinds of features:
    - interaction features
    - squared features

In [ ]:
''' engineer higher-order features '''
# add squares and interactions for continuous features
highords = []
for col in contFeatures:
    # squares
    highords.append(col+'_sq')
    data[highords[-1]] = data[col]**2
    # interactions
    for col2 in res['feature_names']:
        # > test to ensure we don't get both A_B and B_A
        if col2 > col:
            highords.append(col+'_'+col2)
            data[highords[-1]] = data[col]*data[col2]
    
contFeatures = contFeatures + highords
# reorder columns
data = data[contFeatures+cateFeatures+[response]]
# talk
display(data.head())

#### Observations:
- We created all interactions and all squared features, but all are probably not necessary; likely necessary higher-order features probably include:
    - **LSTAT** squared
    - **DIS** with **NOX**
    - **LSTAT** with **RM**
    - **INDUS** squared
    - **AGE** with **NOX**

#### Explanation:
- Some modeling techniques work better if all features are on the same scale, and it also helps with model interpretation, so we standardize each feature to have a mean of $\mu=0$ and standard deviation of $\sigma=1$.
- We only have 1 categorical feature (Charles River flag) which is already binary, but in general, categorical features need to be encoded as binaries.

In [ ]:
''' encode features '''
# standardize continuous features
colTran = ColumnTransformer(transformers=[('scaler', StandardScaler(), contFeatures)], remainder='passthrough', n_jobs=-1)
dataS = pd.DataFrame(data=colTran.fit_transform(data), columns=contFeatures+cateFeatures+[response])

# encode CHAS categorical - already binary, so make integers
dataS['CHAS'] = dataS['CHAS'].astype(int)

# talk
display(dataS.head())
display(dataS.describe())

#### Explanation:
- Feature engineering created many features which are likely not useful.
- We use a model-based feature selection procedure to get a subset of these for modeling; at least a specified number of features will be selected.
- 5-fold validation is employed to minimize any impact of noise on feature selection; final feature importances are averaged over all validation sets.
<br><img src="./Selection_103.webp" alt="5-fold" width="300" height="300"><br>

In [ ]:
''' use cross-validated recursive feature eliminiation to select features '''
# setup
estim = LinearRegression()
minFeatures = 10
cv = 5

# run the procedure
features = contFeatures+cateFeatures
featureSelector = RFECV(estimator=estim, min_features_to_select=minFeatures, cv=cv, n_jobs=-1)
featureSelector.fit(X=dataS[features], y=dataS[response])

# selected features
features = [f for (s, f) in zip(featureSelector.support_, features) if s]
p = len(features)

# unselected raw features
print('Input Features not Used: %s'%([f for f in res['feature_names'] if f not in features]))

# variable uses
print('Response Variable: %s'%response)
print('%d Potential Predictive Features: %s'%(p, features))

#### Observations:
- Statisticians would generally say that if a model includes a higher-order feature, any related raw features should also be included; it's uncertain if this is really necessary.
- According to this preference, unselected features which would need to be included are: **TAX**, **RAD**, **LSTAT**.

### *Success Looks Like: The data has been thoroughly explored, with relevant characteristics documented, with any implications for analysis. Errors, outliers, missing values have been identified and handled. Features have been appropriately prepared for analysis.*

## Analyze - form hypothesis, select analytical techniques, build model(s), assess results
### Form Hypothesis: design experiment using domain knowledge where available, select response variable, select features to utilize

<a href=#top>Go to top</a>
<a id=analyze-hypothesize></a>

**Hypothesis**: there is an identifiable relationship between median house price in Boston and at least one of the gathered data features or their higher-order values

### Select Analytical Technique(s): continuous vs. categorical data, supervised vs. unsupervised learning, clustering vs. classification vs. regression techniques; select method types: optimization, sampling, simple vs. ensemble model, robustness

<a href=#top>Go to top</a>
<a id=analyze-select></a>

The reponse variable is continuously-valued, which indicates that supervised learning, using a regression-based model is appropriate. For demonstrative purposes, three models will be shown:
- Multiple Linear Regression
- Decision Tree Regression
- Simple Neural Network

### Build Model(s): partition data for training / testing / evaluation, hyperparameter tuning, model training

<a href=#top>Go to top</a>
<a id=analyze-build></a>

#### Explanation:
- For model evaluation and hyperparameter tuning, use we cross-validation.
    - The dataset is split randomly into 70% training, 30% validation, 10 times independently.
    - Model metrics are then computed on the validation sets and averaged over them all.
- This helps select a model which generalizes to other data (from the same or similar data-generating process), rather than just perfectly fitting the patterns + random noise in the training data.

In [ ]:
''' cross validation '''
# setup
prng = 42
trainPer = 0.7
cvCnt = 10

# generate the cv indices
#splitter = StratifiedShuffleSplit(n_splits=cvCnt, train_size=trainPer, random_state=prng)
splitter = ShuffleSplit(n_splits=cvCnt, train_size=trainPer, random_state=prng)
cvs = [(tranIndx, testIndx) for (tranIndx, testIndx) in splitter.split(y=dataS['CHAS'].values, X=np.zeros(n))]

#### Multiple Linear Regression
- Linear regression doesn't really have any hyperparameters to tune.
- Acccordingly, we just fit the model with cross-validation.
- Predictions are generated for each validation set to be used later by the model evaluation metrics.

<a href=#top>Go to top</a>
<a id=linreg></a>

In [ ]:
''' linear regression '''
# fit the model and make predictions
linRegResults = [None]*cvCnt
for (indx, (trn, tst)) in enumerate(cvs):
    print('Training for Split %d of %d'%(indx+1, cvCnt))
    # model object
    linReg = LinearRegression(n_jobs=-1)
    # fit
    linReg.fit(X=dataS.loc[trn, features], y=dataS.loc[trn, response])
    # predict on training & testing sets
    trnPred = linReg.predict(X=dataS.loc[trn, features])
    tstPred = linReg.predict(X=dataS.loc[tst, features])
    # save stuff
    linRegResults[indx] = [linReg, trnPred, tstPred] 

#### Decision Tree Regression
- The decision tree regressor is a form of nonlinear regression.
- There are several tunable parameters; three are tuned here:
    - splitting criterion - changes the loss function used
    - maximum depth - deeper means more complex, and increases risk of overfitting
    - minimum samples per leaf - few samples per leave increases risk of overfitting
- Predictions are generated for each validation set to be used later by the model evaluation metrics.

<a href=#top>Go to top</a>
<a id=dectree></a>

In [ ]:
''' decision tree regression with hyperparameter tuning '''
# setup the parameter grid
params = {'criterion':['squared_error', 'absolute_error'], 'max_depth':[None, 5, 10], 'min_samples_leaf':[None, 5, 10, 0.05, 0.1]}

# perform hyperparameter tuning and model training
GSC = GridSearchCV(estimator=DecisionTreeRegressor(), param_grid=params, cv=cvs, n_jobs=-1, verbose=4)
GSC.fit(X=dataS[features], y=dataS[response])
print('Best Parameters: %s'%GSC.best_params_)

# get the predictions for all cvs, using the best set of parameters
decTreeResults = [None]*cvCnt
for (indx, (trn, tst)) in enumerate(cvs):
    print('Training for Split %d of %d'%(indx+1, cvCnt))
    # model object
    decTree = deepcopy(GSC.best_estimator_)
    # fit
    decTree.fit(X=dataS.loc[trn, features], y=dataS.loc[trn, response])
    # predict on training & testing sets
    trnPred = decTree.predict(X=dataS.loc[trn, features])
    tstPred = decTree.predict(X=dataS.loc[tst, features])
    # save stuff
    decTreeResults[indx] = [decTree, trnPred, tstPred] 

#### Simple Neural Network
- A neural network model can generally match complex nonlinear relationships.
- It requires many parameters and more training data / time than simpler models.
- We just demonstrate it here with 1 cross-validation set.
- Neural networks are very powerful, but *not a panacea*.

<a href=#top>Go to top</a>
<a id=nn></a>

In [ ]:
''' setup for data access '''
# now make the datasets & dataloaders
batchSize = 64

# make train and test sets
cvIndex = 1
trn = cvs[cvIndex][0]; tst = cvs[cvIndex][1]
trnX = dataS[features].values[trn,:]
tstX = dataS[features].values[tst,:]
trnY = np.atleast_2d(dataS[response].values[trn]).T
tstY = np.atleast_2d(dataS[response].values[tst]).T

# training data is accessed in batches, testing data is not
trainData = Data(trnX, trnY)
trainLoad = DataLoader(dataset=trainData, batch_size=batchSize, shuffle=True)
testData = Data(tstX, tstY)
testLoad = DataLoader(dataset=testData, batch_size=len(testData))

In [ ]:
''' define the model & operating parameters '''
# define the modeling parameters
layers = (p, 16, 32, 64, 32, 16, 8, 1) # input layer size, hidden layer sizes, output layer size
activations = (None, 'tanh', 'tanh', 'tanh', 'tanh', 'tanh', 'tanh', 'relu') # first element should be None; it's for the input layer
pDropout = 0.2
weightInit = 'kai'

# set the learning rate
learningRate = 0.01
stepSize = 50
stepMult = 0.1

# setup the network, optimizer, and loss
torch.manual_seed(prng)
thisNN = myNN(layers, weightInit, pDropout, activations)
print(thisNN)
print('%d parameters'%np.sum([len(p) for p in thisNN.parameters()]))
#thisNN.show()
optimizer = torch.optim.SGD(params=thisNN.parameters(), lr=learningRate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=stepSize, gamma=stepMult)
loss = nn.MSELoss()

# define the tensorboard writer
writr = SummaryWriter(log_dir='./neuralnet/')

In [ ]:
''' train the neural network '''
# setup
epochs = 1000
talkFreq = 0.1

# untrained performance
thisNN.train(False)
print('Pre-training Loss = %0.2f'%loss(thisNN(testLoad.dataset.x), testLoad.dataset.y).detach().numpy())

# train
trnLoss, tstLoss  = trainNN([trainLoad, testLoad], thisNN, loss, optimizer, scheduler, epochs, writr, None, talkFreq)
#thisNN.show()

# get the predictions
nnPred = thisNN(torch.FloatTensor(dataS[features].values)).detach().numpy()

### Assess Results: Numerical evaluation with appropriate metrics, evaluation of model results vs. known responses, feature importance, detailed error (residual) analysis, identification of spurious correlation, confounding features, and need for additional data, comparison to simpler / other / no models, domain-specific assessments,  evaluation of substantiality vis-a-vis business needs; preparation of publication-ready statistical tables and result visualizations

<a href=#top>Go to top</a>
<a id=analyze-assess></a>

#### Explanation:
- We evaluate several metrics:
    - mean squared error: MSE is often used to measure regression model fits
    - mean absolute percentage error: MAPE is easily interpretable as the average error relative to average response value
    - r squared: $r^2$ measures the linear correlation between actual response and predicted values, closer to 1.0 is better
    - root mean squared error: RMSE is the square root of the MSE; it is easily interpretable, as the units of error is the same as the units of the response
- All metrics here should be minimized, except for $r^2$, which should be maximized.
- Other choices for metrics could be made.
- While we measure all metrics, RMSE is taken as the *primary* metric.

In [ ]:
''' review error metrics '''
# metrics
mainMetric = 'rmse' # this is the most important metric - will use later
metrics = {'mse':mean_squared_error, 'mape':mean_absolute_percentage_error, 'r2':r2_score, 'rmse':RMSE}
template = dict.fromkeys(metrics.keys(), np.nan)
resultsTranLR = {}
resultsTestLR = {}
resultsTranDT = {}
resultsTestDT = {}

# compute all metrics
for indx in range(len(cvs)):
    # linear Regression
    resKeyLR = 'Linear Regression %d'%indx
    resultsTranLR[resKeyLR] = deepcopy(template)
    resultsTestLR[resKeyLR] = deepcopy(template)
    # Decision Tree
    resKeyDT = 'Decision Tree %d'%indx    
    resultsTranDT[resKeyDT] = deepcopy(template)
    resultsTestDT[resKeyDT] = deepcopy(template)
    for metric in metrics.keys():
        # linear regression
        resultsTranLR[resKeyLR][metric] = metrics[metric](y_true=dataS[response].values[cvs[indx][0]], y_pred=linRegResults[indx][1])
        resultsTestLR[resKeyLR][metric] = metrics[metric](y_true=dataS[response].values[cvs[indx][1]], y_pred=linRegResults[indx][2])
        # decision tree
        resultsTranDT[resKeyDT][metric] = metrics[metric](y_true=dataS[response].values[cvs[indx][0]], y_pred=decTreeResults[indx][1])
        resultsTestDT[resKeyDT][metric] = metrics[metric](y_true=dataS[response].values[cvs[indx][1]], y_pred=decTreeResults[indx][2])
        
# put all together as some dataframes
metricsLR = pd.DataFrame(resultsTranLR).T.join(pd.DataFrame(resultsTestLR).T, lsuffix='_train', rsuffix='_test')
metricsLR = pd.concat([metricsLR, pd.DataFrame(metricsLR.min(), columns=['Linear Regression Min']).T,
                       pd.DataFrame(metricsLR.mean(), columns=['Linear Regression Mean']).T,
                       pd.DataFrame(metricsLR.max(), columns=['Linear Regression Max']).T])
metricsDT = pd.DataFrame(resultsTranDT).T.join(pd.DataFrame(resultsTestDT).T, lsuffix='_train', rsuffix='_test')
metricsDT = pd.concat([metricsDT, pd.DataFrame(metricsDT.min(), columns=['Decision Tree Min']).T,
                       pd.DataFrame(metricsDT.mean(), columns=['Decision Tree Mean']).T,
                       pd.DataFrame(metricsDT.max(), columns=['Decision Tree Max']).T])

# talk
print('Linear Regression Metrics')
display(metricsLR)
print('Decision Tree Metrics')
display(metricsDT)

#### Observations:
- The range of RMSEs for *Linear Regression* overlaps with that for the *Decision Tree* model.
- The average RMSE for *Linear Regression* is less than that for the *Decision Tree* model (same for all metrics).
- Considering RMSEs, I would select *Linear Regression* as the best model type.

#### Explanation:
An important part of modeling is evaluating how important each feature is, especially relative to other features:
- For regression models, the model coefficients can be interpreted as how much the response should change for a unit change in each feature.
- If features are normalized to the same scale, linear regression coefficients are also interpretable as feature importances.
- Decision Trees infer feature importance as a function of the decrease in node impurity (for a node that is split based on each feature) and the probability of reaching that node.
- More generally, Shapley values can attribute model performance to features in an additive way. The idea of Shapley values comes from cooperative game theory, in which the gain generated by a group of actors is distributed among them.

In assessing and diagnosing regression models, there are several standard visualization tools we review:
- all are based on predicted values and / or residuals (difference between actual responses and predictions)
- *Actual Response Values (X) vs. Predictions*: would like to see a positive linear correlation, with the datapoints narrowly dispersed around the fit line
- *Histogram of Residuals*: linear regression assumes residuals are iid Gaussian white noise with constant variance
- *Sequence / Time (X) vs. Residuals*: any pattern would suggest there is a time or sequence pattern in the response values, which has not been accounted for
- *Actual Response Values (X) vs. Residuals*: would like to see the same dispersion of residuals no matter how large / small the actual response values are

In [ ]:
''' best linear regression '''
# best important metric value
vals = metricsLR[mainMetric+'_test'][:cvCnt].values
bestIndex = np.argmin(vals)
print('Best Linear Regression Model Test %s = %0.2f'%(mainMetric, vals[bestIndex]))

# model coefficients
bestModel = linRegResults[bestIndex][0]
coefs = pd.DataFrame(data=bestModel.coef_, index=features, columns=['Coefficient'])
coefs['abs'] = coefs['Coefficient'].abs()
coefs.sort_values(by='abs', ascending=False, inplace=True)
coefs = pd.concat([pd.DataFrame(index=['Intercept'], data={'Coefficient':[bestModel.intercept_]}), coefs.drop(columns=['abs'])])
display(coefs)

# prep the dataframe of actuals & predicteds
predRes = pd.DataFrame(dataS[response], columns=[response])
predRes.loc[cvs[bestIndex][0], 'color'] = 'red'
predRes.loc[cvs[bestIndex][1], 'color'] = 'green'
predRes.loc[cvs[bestIndex][0], 'Prediction'] = linRegResults[bestIndex][1]
predRes.loc[cvs[bestIndex][1], 'Prediction'] = linRegResults[bestIndex][2]
predRes['Error'] = predRes[response] - predRes['Prediction']

# see the model results plot
fig = ResultsPlots(predRes, None, response, 'Prediction', 'Error', 'color', 'Best Linear Regression Model Result - Test %s = %02f'%(mainMetric, vals[bestIndex]))
plyoff.iplot(fig)#, include_mathjax='cdn')

In [ ]:
''' best decision tree '''
# best important metric value
vals = metricsDT[mainMetric+'_test'][:cvCnt].values
bestIndex = np.argmin(vals)
print('Best Decision Tree Model Test %s = %0.2f'%(mainMetric, vals[bestIndex]))

# model coefficients
bestModel = decTreeResults[bestIndex][0]
featImports = pd.DataFrame(data=bestModel.feature_importances_, index=features, columns=['Importance']).sort_values(by='Importance', ascending=False)
display(featImports)

# prep the dataframe of actuals & predicteds
predRes = pd.DataFrame(dataS[response], columns=[response])
predRes.loc[cvs[bestIndex][0], 'color'] = 'red'
predRes.loc[cvs[bestIndex][1], 'color'] = 'green'
predRes.loc[cvs[bestIndex][0], 'Prediction'] = decTreeResults[bestIndex][1]
predRes.loc[cvs[bestIndex][1], 'Prediction'] = decTreeResults[bestIndex][2]
predRes['Error'] = predRes[response] - predRes['Prediction']

# see the model results plot
fig = ResultsPlots(predRes, None, response, 'Prediction', 'Error', 'color', 'Best Decision Tree Model Result - Test %s = %02f'%(mainMetric, vals[bestIndex]))
plyoff.iplot(fig)#, include_mathjax='cdn')

print('Best Tree')
res = plot_tree(decTreeResults[bestIndex][0])

In [ ]:
''' Neural Network '''
# best important metric value
nnRMSE = metrics[mainMetric](testLoad.dataset.y, thisNN(testLoad.dataset.x).detach().numpy())
print('Neural Network Test %s = %0.2f'%(mainMetric, nnRMSE))

# feature importances
att = DeepLift(thisNN)
#att = IntegratedGradients(thisNN)
att = NoiseTunnel(att)
baseline = torch.zeros(testLoad.dataset.x.shape[0], testLoad.dataset.x.shape[1]) 
attributions, delta = att.attribute(inputs=testLoad.dataset.x, baselines=baseline, target=0, return_convergence_delta=True)
featImports = pd.DataFrame(np.mean(attributions.detach().numpy(), axis=0).T, index=features, columns=['Importance'])
featImports['abs'] = featImports['Importance'].abs()
featImports.sort_values(by='abs', ascending=False, inplace=True)
featImports.drop(columns=['abs'], inplace=True)
display(featImports)

# prep the dataframe of actuals & predicteds
predRes = pd.DataFrame(dataS[response], columns=[response])
predRes.loc[cvs[0][0], 'color'] = 'red'
predRes.loc[cvs[0][1], 'color'] = 'green'
predRes['Prediction'] = nnPred
predRes['Error'] = predRes[response] - predRes['Prediction']

# see the model results plot
fig = ResultsPlots(predRes, None, response, 'Prediction', 'Error', 'color', 'Neural Network Model Result - Test %s = %02f'%(mainMetric, nnRMSE))
plyoff.iplot(fig)#, include_mathjax='cdn')

#### Observations:
Importances:
- All three also models show some significant differences in most important features - but also similarities.
- *Linear Regression*: **PTRATIO_TAX**, **RM**, **AGE**
- *Decision Tree*: **RM**, **AGE_LSTAT**, **LSTAT_sq**
- *Neural Network*: **LSTAT_TAX**, **RM**, **AGE_LSTAT**

Diagnostic Plots:
- All three of these models have issues showing in residual plots, which would need to be evaluated.
- *Linear Regression*: strange vertical line of predictions & residuals at the maximum response value; suggests some response value data truncation
- *Decision Tree*: horizontal lines in 2 plots indicate the decision tree might be overly simple, as ranges of responses have the same predictions
- *Neural Network*: odd patterns visible

### *Success Looks Like: A relevant and testable hypothesis has been formed and analyzed using the appropriate modeling techniques. Model results have been assessed using appropriate metrics.*

## Report - Document the APA portion of the BAPARA process followed as relevant to audience and context, assumptions made and the degree to which they were validated (or not), shortcomings in the data, and how they may be overcome, potential ways to improve the result, generalizability of the model(s), importance & value to the business; this is when peer review should be performed
- Note that a negative result - i.e. a model that does not perform good enough - should still be reported. Potential value remains in how the process was followed (code, data), how models were run, and the results.
- *Success Looks Like: Assumptions, decisions, process, results, and implications accurately and effectively documented.* 

## Act - Realize the value proposition of the project by applying the insight in the workflow, evaluate effectiveness in context and identify the source of the ultimate value; if operationalization is needed, this can be almost as involved as the entire APA portion of the BAPARA process

<a href=#top>Go to top</a>
<a id=report-act></a>

## The end

<a href=#top>Go to top</a>
<a id=end></a>